# bytes包与字节串操作
我相信，经过上一次的学习，你已经对`strings.Builder`和`strings.Reader`这两个类型足够熟悉了。

我上次还建议你去自行查阅`strings`代码包中的其他程序实体。如果你认真去看了，那么肯定会对我们今天要讨论的`bytes`代码包，有种似曾相识的感觉。

## 前导内容：`bytes.Buffer`基础知识
`strings`包和`bytes`包可以说是一对孪生兄弟，它们在API方面非常的相似。单从它们提供的函数的数量和功能上讲，差别可以说是微乎其微。

“从 API 方面”，简单来说就是指 “你调用函数的方式”（包括函数的名字、传参的顺序、返回值的类型等）。

**只不过，`strings`包主要面向的是`Unicode`字符和经过`UTF-8`编码的字符串，而`bytes`包面对的则主要是字节和字节切片**。

我今天会主要讲`bytes`包中最有特色的类型`Buffer`。顾名思义，**`bytes.Buffer`类型的用途主要是作为字节序列的缓冲区**。

与`strings.Builder`类型一样，`bytes.Buffer`也是开箱即用的。

但不同的是，`strings.Builder`只能拼接和导出字符串，而`bytes.Buffer`不但可以拼接、截断其中的字节序列，以各种形式导出其中的内容，还可以顺序地读取其中的子序列。

可以说，`bytes.Buffer`是集读、写功能于一身的数据类型。当然了，这些也基本上都是作为一个缓冲区应该拥有的功能。

在内部，`bytes.Buffer`类型同样是使用字节切片作为内容容器的。并且，与`strings.Reader`类型类似，`bytes.Buffer`有一个int类型的字段，用于代表已读字节的计数，可以简称为已读计数。

不过，这里的已读计数就无法通过`bytes.Buffer`提供的方法计算出来了。

我们先来看下面的代码：

In [ ]:
var buffer1 bytes.Buffer
contents := "Simple byte buffer for marshaling data."
fmt.Printf("Writing contents %q ...\n", contents)
buffer1.WriteString(contents)
fmt.Printf("The length of buffer: %d\n", buffer1.Len())
fmt.Printf("The capacity of buffer: %d\n", buffer1.Cap())

我先声明了一个`bytes.Buffer`类型的变量`buffer1`，并写入了一个字符串。然后，我想打印出这个`bytes.Buffer`类型的值（以下简称`Buffer`值）的长度和容量。在运行这段代码之后，我们将会看到如下的输出：

In [ ]:
Writing contents "Simple byte buffer for marshaling data." ...
The length of buffer: 39
The capacity of buffer: 64

乍一看这没什么问题。长度39和容量64的含义看起来与我们已知的概念是一致的。我向缓冲区中写入了一个长度为39的字符串，所以`buffer1`的长度就是39。

根据切片的自动扩容策略，64这个数字也是合理的。另外，可以想象，这时的已读计数的值应该是0，这是因为我还没有调用任何用于读取其中内容的方法。

可实际上，与`strings.Reader`类型的Len方法一样，buffer1的Len方法返回的也是内容容器中未被读取部分的长度，而不是其中已存内容的总长度（以下简称内容长度）。示例如下：

In [ ]:
p1 := make([]byte, 7)
n, _ := buffer1.Read(p1)
fmt.Printf("%d bytes were read. (call Read)\n", n)
fmt.Printf("The length of buffer: %d\n", buffer1.Len())
fmt.Printf("The capacity of buffer: %d\n", buffer1.Cap())

当我从`buffer1`中读取一部分内容，并用它们填满长度为7的字节切片p1之后，`buffer1`的`Len`方法返回的结果值也会随即发生变化。如果运行这段代码，我们会发现，这个缓冲区的长度已经变为了`32`。

另外，因为我们并没有再向该缓冲区中写入任何内容，所以它的容量会保持不变，仍是`64`。

总之，在这里，你需要记住的是，**`Buffer`值的长度是未读内容的长度，而不是已存内容的总长度。** 它与在当前值之上的读操作和写操作都有关系，并会随着这两种操作的进行而改变，它可能会变得更小，也可能会变得更大。

而`Buffer`值的容量指的是它的**内容容器（也就是那个字节切片）的容量**，它只与在当前值之上的写操作有关，并会随着内容的写入而不断增长。

再说已读计数。由于`strings.Reader`还有一个`Size`方法可以给出内容长度的值，所以我们用内容长度减去未读部分的长度，就可以很方便地得到它的已读计数。

然而，`bytes.Buffer`类型却没有这样一个方法，它只有`Cap`方法。可是`Cap`方法提供的是**内容容器的容量，也不是内容长度。**

并且，这里的内容容器容量在很多时候都与内容长度不相同。因此，没有了现成的计算公式，只要遇到稍微复杂些的情况，我们就很难估算出`Buffer`值的已读计数。

一旦理解了已读计数这个概念，并且能够在读写的过程中，实时地获得已读计数和内容长度的值，我们就可以很直观地了解到当前Buffer值各种方法的行为了。不过，很可惜，这两个数字我们都无法直接拿到。

虽然，我们无法直接得到一个`Buffer`值的已读计数，并且有时候也很难估算它，但是我们绝对不能就此作罢，而应该通过研读`bytes.Buffer`和文档和源码，去探究已读计数在其中起到的关键作用。

否则，我们想用好`bytes.Buffer`的意愿，恐怕就不会那么容易实现了。

下面的这个问题，如果你认真地阅读了`bytes.Buffer`的源码之后，就可以很好地回答出来。

**我们今天的问题是：bytes.Buffer类型的值记录的已读计数，在其中起到了怎样的作用？**

这道题的典型回答是这样的。

`bytes.Buffer`中的已读计数的大致功用如下所示。

- 读取内容时，相应方法会依据已读计数找到未读部分，并在读取后更新计数。
- 写入内容时，如需扩容，相应方法会根据已读计数实现扩容策略。
- 截断内容时，相应方法截掉的是已读计数代表索引之后的未读部分。
- 读回退时，相应方法需要用已读计数记录回退点。
- 重置内容时，相应方法会把已读计数置为0。
- 导出内容时，相应方法只会导出已读计数代表的索引之后的未读部分。
- 获取长度时，相应方法会依据已读计数和内容容器的长度，计算未读部分的长度并返回。

## 问题解析
通过上面的典型回答，我们已经能够体会到已读计数在`bytes.Buffer`类型，及其方法中的重要性了。没错，`bytes.Buffer`的绝大多数方法都用到了**已读计数**，而且都是非用不可。

在读取内容的时候，相应方法会先根据已读计数，判断一下内容容器中是否还有未读的内容。如果有，那么它就会从已读计数代表的索引处开始读取。

在读取完成后，它还会及时地更新已读计数。也就是说，它会记录一下又有多少个字节被读取了。这里所说的相应方法包括了所有名称以`Read`开头的方法，以及`Next`方法和`WriteTo`方法。

在写入内容的时候，绝大多数的相应方法都会先检查当前的内容容器，是否有足够的容量容纳新的内容。如果没有，那么它们就会对内容容器进行扩容。

在扩容的时候，方法会在必要时，依据已读计数找到未读部分，并把其中的内容拷贝到扩容后内容容器的头部位置。

然后，方法将会把已读计数的值置为`0`，以表示下一次读取需要从内容容器的第一个字节开始。用于写入内容的相应方法，包括了所有名称以`Write`开头的方法，以及`ReadFrom`方法。

用于截断内容的方法`Truncate`，会让很多对`bytes.Buffer`不太了解的程序开发者迷惑。 它会接受一个`int`类型的参数，这个参数的值代表了：在截断时需要保留头部的多少个字节。

**不过，需要注意的是，这里说的头部指的并不是内容容器的头部，而是其中的未读部分的头部。**头部的起始索引正是由已读计数的值表示的。因此，在这种情况下，已读计数的值再加上参数值后得到的和，就是内容容器新的总长度。

在`bytes.Buffer`中，用于读回退的方法有`UnreadByte`和`UnreadRune`。 这两个方法分别用于`回退一个字节`和`回退一个Unicode字符`。调用它们一般都是为了退回在上一次被读取内容末尾的那个分隔符，或者为重新读取前一个字节或字符做准备。

不过，**退回的前提是，在调用它们之前的那一个操作必须是“读取”，并且是成功的读取，否则这些方法就只能忽略后续操作并返回一个非nil的错误值**。

`UnreadByte`方法的做法比较简单，把已读计数的值减1就好了。而`UnreadRune`方法需要从已读计数中减去的，是上一次被读取的Unicode字符所占用的字节数。

这个字节数由`bytes.Buffer`的另一个字段负责存储，它在这里的有效取值范围是[1, 4]。只有`ReadRune`方法才会把这个字段的值设定在此范围之内。

由此可见，只有紧接在调用`ReadRune`方法之后，对`UnreadRune`方法的调用才能够成功完成。该方法明显比`UnreadByte`方法的适用面更窄。

我在前面说过，`bytes.Buffer`的Len方法返回的是内容容器中未读部分的长度，而不是其中已存内容的总长度（即：内容长度）。

而该类型的`Bytes`方法和`String`方法的行为，与`Len`方法是保持一致的。前两个方法只会去访问未读部分中的内容，并返回相应的结果值。

在我们剖析了所有的相关方法之后，可以这样来总结：**在已读计数代表的索引之前的那些内容，永远都是已经被读过的，它们几乎没有机会再次被读取**。

不过，这些已读内容所在的内存空间可能会被存入新的内容。这一般都是由于重置或者扩充内容容器导致的。这时，已读计数一定会被置为0，从而再次指向内容容器中的第一个字节。这有时候也是为了避免内存分配和重用内存空间。

## 总结
总结一下，`bytes.Buffer`是一个集读、写功能于一身的数据类型。它非常适合作为字节序列的缓冲区。我们会在下一篇文章中继续对bytes.Buffer的知识进行延展。如果你对于这部分内容有什么样问题，欢迎给我留言，我们一起讨论。

## 关于buffer中已读计数的解释

这是一个直击灵魂的问题！你发现了一个非常有趣的矛盾：

> 图片里列出了那么多基于“已读计数”的复杂逻辑，但在 `bytes.Buffer` 的公开方法里，我们却根本找不到 `ReadCount` 或者 `Offset` 这样的方法。

**它是怎么做到的？**

答案非常简单粗暴：**它藏了一手。**

虽然它没有公开告诉你，但在 `bytes.Buffer` 的源代码内部，有一个**私有字段（Private Field）** 专门用来干这件事。

### 1\. 揭秘幕后黑手：`off` 字段

如果我们打开 Go 语言 `bytes.Buffer` 的源码（`src/bytes/buffer.go`），你会看到它的结构体定义其实是这样的（简化版）：

```go
type Buffer struct {
    buf      []byte // 底层真正的字节切片
    off      int    // <--- 就是它！已读计数 (read offset)
    lastRead readOp // 用于回退的记录
}
```

看到那个 **`off`** 了吗？这就是你心心念念的“已读计数”。

  * **它是私有的**（首字母小写）：所以你在外面通过 `b.off` 访问不到，也没有 `b.Offset()` 方法能获取它。
  * **它是核心**：图片里提到的所有逻辑，全都是内部代码在这个 `off` 变量上做加减法。

-----

### 2\. 它如何解决你图片里的那些问题？

既然我们知道有一个隐形的 `off` 变量，我们再回头看你发的图片，一切就豁然开朗了。我来为你一一拆解它是如何“解决”的：

#### A. 关于“获取长度” (Len)

你看到的公开方法 `Len()`，其实是个**假象**。它并不是直接返回切片的长度，而是算出来的。

**内部逻辑：**

```go
func (b *Buffer) Len() int {
    return len(b.buf) - b.off  // 总长度 - 已读计数
}
```

所以，对外你看到的是“剩余未读长度”，对内它是靠 `off` 算出来的。

#### B. 关于“读取内容”

当你调用 `Read()` 时，它利用 `off` 来切分数据。

**内部逻辑：**

1.  拿出 `b.buf[b.off:]` 这部分数据给你（从已读计数开始切）。
2.  你读了 `n` 个字节。
3.  它执行 `b.off += n`（悄悄更新已读计数）。

#### C. 关于“写入内容”时的“扩容策略” (最精彩的部分)

这对应你图片里的第二点。当你要写入新数据，而后面空间不够时，`Buffer` 会先检查 `off`。

**场景：**
假设 `buf` 容量是 10，内容是 `[A, B, C, D, E, _, _, _, _, _]`。

1.  你读了 3 个：`off` 变成了 3。
2.  虽然看起来前面有 3 个空位，但 `buf` 的末尾空间不够了。
3.  **Buffer 的解决办法**：它发现 `off > 0`，说明前面有废弃空间。它不会急着申请新内存，而是执行 **`copy`（平移）** 操作，把数据往前挪，然后把 `off` **归零**。

#### D. 关于“重置内容”

当你调用 `Reset()` 时：

**内部逻辑：**

```go
func (b *Buffer) Reset() {
    b.buf = b.buf[:0] // 切片长度归零
    b.off = 0         // <--- 已读计数归零！
}
```

它并不清空内存里的真实数据，只是把这个“计数器”拨回 0，假装一切重新开始。

-----

### 3\. 为什么不把 `off` 公开给我们用？

你可能会问：“既然 `off` 这么有用，为什么像 `strings.Reader` 那样提供一个 `Size()` 或 `Seek()` 让我们访问它呢？”

这是由 `bytes.Buffer` 的**定位**决定的：

1.  **它是一个流（Stream）**：流的特性是“过了这个村就没这个店”。Buffer 的设计哲学是让你一直往前读写，而不是跳来跳去。
2.  **封装复杂性**：如果公开了 `off`，允许你随意修改（比如像 `Seek` 一样），那么 Buffer 内部那个\*\*“自动把数据往前挪以节省内存”\*\*的机制就会乱套。Go 团队决定把这个复杂的内存管理逻辑封装在黑盒子里，只给你简单好用的 `Read/Write` 接口。

### 总结

你发的那张图里的所有功能，都是通过**私有字段 `off`** 配合 **切片操作 (`buf[off:]`)** 来实现的。

  * **对外**：它只暴露 `Len()`（剩余量），装作没有“已读计数”这回事。
  * **对内**：它通过精细地控制 `off`，实现了数据的滑动、丢弃和复用。

# 下
上一篇文章中我们已经知道了`bytes.Buffer`中已读计数的大致作用，并围绕进行了展开，下面我们来进行相关的知识扩展。
## 知识扩展

**问题 1：bytes.Buffer的扩容策略是怎样的？**
`Buffer`值既可以被手动扩容，也可以进行自动扩容。并且，这两种扩容方式的策略是基本一致的。所以，除非我们完全确定后续内容所需的字节数，否则让`Buffer`值自动去扩容就好了。

在扩容的时候，`Buffer`值中相应的代码（以下简称扩容代码）会先判断内容容器的剩余容量，是否可以满足调用方的要求，或者是否足够容纳新的内容。

如果可以，那么扩容代码会在当前的内容容器之上，进行长度扩充。

更具体地说，**如果内容容器的容量与其长度的差，大于或等于另需的字节数，那么扩容代码就会通过切片操作对原有的内容容器的长度进行扩充**，就像下面这样：

In [ ]:
b.buf = b.buf[:length + need]
// 此处有点像把水杯的刻度线往上提了一点点，表示现在杯子里多了点水，杯子容量没变，只是还够，不用换杯子。
// 并非物理上的扩容，而是逻辑上的延申

**反之，如果内容容器的剩余容量不够了，那么扩容代码可能就会用新的内容容器去替代原有的内容容器，从而实现扩容**。

不过，这里还有一步优化。

**如果当前内容容器的容量的一半，仍然大于或等于其现有长度（即未读字节数）再加上另需的字节数的和**，
为什么要满足 cap/2 >= ... 这个条件？
你可能会问：“只要总空闲空间（已读+尾部）够用，为什么不直接搬运？为什么要加一个‘容量的一半’这么严格的限制？”

这涉及到一个核心的性能权衡：避免“抖动”（Thrashing）。

理由一：搬运是有成本的
将数据从数组中间拷贝到头部（Memory Copy）是需要消耗 CPU 资源的。如果我们每次空间稍微不够就搬运，CPU 开销会很大。

理由二：防止“刚搬完又要扩容”
假设没有这个 1/2 的限制，只要能塞进去就搬运：

搬运： 费力把数据搬到头部。

写入： 写入新数据，把 Buffer 填得很满（比如填到了 95%）。

再写： 下一次用户再想写一点点数据，发现连头部哪怕重整了也塞不下了。

扩容： 最终还是不得不申请新内存进行物理扩容。

在这种情况下，第 1 步的“搬运”就成了无用功。因为反正马上就要扩容（物理扩容也包含搬运），不如直接跳过重整，直接扩容。

理由三：保证“蓄水池”效应
cap/2 >= b.Len() + need 这个公式翻译成人话就是：

**“只有当搬运并写入新数据后，Buffer 的占用率仍然小于 50% 时，我才愿意费力气去搬运。”**

这意味着，一旦我执行了这次搬运操作，我能保证：

Buffer 里至少还有**一半以上**的空地。

后续的多次写入操作可以直接追加，**很长一段时间**内都不需要再次搬运或扩容。
即：

In [ ]:
cap(b.buf)/2 >= b.Len() + need

那么，扩容代码就会复用现有的内容容器，并把容器中的未读内容拷贝到它的头部位置。

这也意味着其中的已读内容，将会全部被未读内容和之后的新内容覆盖掉。

这样的复用预计可以至少节省掉一次后续的扩容所带来的内存分配，以及若干字节的拷贝。

**若这一步优化未能达成**，也就是说，当前内容容器的容量小于新长度的二倍。

那么，扩容代码就只能再创建一个新的内容容器，并把原有容器中的未读内容拷贝进去，最后再用新的容器替换掉原有的容器。这个新容器的容量将会等于原有容量的二倍再加上另需字节数的和。
<pre>
新容器的容量=2*原有容量+所需字节数
</pre>
通过上面这些步骤，对内容容器的扩充基本上就完成了。不过，为了内部数据的一致性，以及避免原有的已读内容可能造成的数据混乱，扩容代码还会把已读计数置为0，并再对内容容器做一下切片操作，以掩盖掉原有的已读内容。

顺便说一下，对于处在零值状态的`Buffer`值来说，如果第一次扩容时的另需字节数不大于`64`，那么该值就会基于一个预先定义好的、长度为`64`的字节数组来创建内容容器。

在这种情况下，这个内容容器的容量就是`64`。这样做的目的是为了让`Buffer`值在刚被真正使用的时候就可以快速地做好准备。

**问题2：bytes.Buffer中的哪些方法可能会造成内容的泄露？**
首先明确一点，什么叫内容泄露？这里所说的内容泄露是指，使用`Buffer`值的一方通过某种非标准的（或者说不正式的）方式，得到了本不该得到的内容。

比如说，我通过调用`Buffer`值的某个用于读取内容的方法，得到了一部分未读内容。我应该，也只应该通过这个方法的结果值，拿到在那一时刻`Buffer`值中的未读内容。

但是，在这个`Buffer`值又有了一些新内容之后，我却可以通过当时得到的结果值，直接获得新的内容，而不需要再次调用相应的方法。

**比喻**：
1. 生活中的类比：复印件 vs. 共享文档
- 理想情况（安全）： 你向我要一份文件的第 1 页。我拿去复印了一张给你。之后我在原文件的第 2 页写了新东西，或者甚至修改了第 1 页，你手里的那张复印件是永远不会变的。这是安全的。

- 实际情况（泄露）： 你向我要一份文件。我为了省事，没有复印，而是直接给了你一把原本的钥匙，说：“你去柜子里看吧”。 你看完了。过了一会儿，我往柜子里的那个本子上写了新内容。虽然你没有再次向我申请，但因为你手里还握着那把钥匙，你偷偷打开柜子，就看到了我刚写的新内容。

**这就是内容泄露。**

这就是典型的非标准读取方式。这种读取方式是不应该存在的，即使存在，我们也不应该使用。因为它是在无意中（或者说一不小心）暴露出来的，其行为很可能是不稳定的。

在`bytes.Buffer`中，`Bytes`方法和`Next`方法都可能会造成内容的泄露。原因在于，**它们都把基于内容容器的切片直接返回给了方法的调用方**。

我们都知道，通过切片，我们可以**直接访问和操纵它的底层数组**。不论这个切片是基于某个数组得来的，还是通过对另一个切片做切片操作获得的，都是如此。

在这里，`Bytes`方法和`Next`方法返回的字节切片，都是通过对内容容器做切片操作得到的。也就是说，它们与内容容器共用了同一个底层数组，起码在一段时期之内是这样的。

以`Bytes`方法为例。它会返回在调用那一刻其所属值中的所有未读内容。示例代码如下：

In [ ]:
contents := "ab"
buffer1 := bytes.NewBufferString(contents)
fmt.Printf("The capacity of new buffer with contents %q: %d\n",
 contents, buffer1.Cap()) // 内容容器的容量为：8。
unreadBytes := buffer1.Bytes()
fmt.Printf("The unread bytes of the buffer: %v\n", unreadBytes) // 未读内容为：[97 98]。

我用字符串值"ab"初始化了一个Buffer值，由变量buffer1代表，并打印了当时该值的一些状态。

你可能会有疑惑，我只在这个Buffer值中放入了一个长度为2的字符串值，但为什么该值的容量却变为了8。

虽然这与我们当前的主题无关，但是我可以提示你一下：你可以去阅读runtime包中一个名叫stringtoslicebyte的函数，答案就在其中。

接着说buffer1。我又向该值写入了字符串值"cdefg"，此时，其容量仍然是8。我在前面通过调用buffer1的Bytes方法得到的结果值unreadBytes，包含了在那时其中的所有未读内容。

但是，由于这个结果值与buffer1的内容容器在此时还共用着同一个底层数组，所以，我只需通过简单的再切片操作，就可以利用这个结果值拿到buffer1在此时的所有未读内容。如此一来，buffer1的新内容就被泄露出来了。

In [ ]:
buffer1.WriteString("cdefg")
fmt.Printf("The capacity of buffer: %d\n", buffer1.Cap()) // 内容容器的容量仍为：8。
unreadBytes = unreadBytes[:cap(unreadBytes)]
fmt.Printf("The unread bytes of the buffer: %v\n", unreadBytes) // 基于前面获取到的结果值可得，未读内容为：[97 98 99 100 101 102 103 0]。

如果我当时把`unreadBytes`的值传到了外界，那么外界就可以通过该值操纵`buffer1`的内容了，就像下面这样：

In [ ]:
unreadBytes[len(unreadBytes)-2] = byte('X') // 'X'的ASCII编码为88。
fmt.Printf("The unread bytes of the buffer: %v\n", buffer1.Bytes()) // 未读内容变为了：[97 98 99 100 101 102 88]。

现在，你应该能够体会到，这里的内容泄露可能造成的严重后果了吧？对于`Buffer`值的`Next`方法，也存在相同的问题。

不过，如果经过扩容，`Buffer`值的内容容器或者它的底层数组被重新设定了，那么之前的内容泄露问题就无法再进一步发展了。我在demo80.go文件中写了一个比较完整的示例，你可以去看一看，并揣摩一下。

## 总结
我们结合两篇内容总结一下。与`strings.Builder`类型不同，`bytes.Buffer`不但可以`拼接、截断`其中的字节序列，以各种形式导出其中的内容，还可以顺序地读取其中的子序列。

`bytes.Buffer`类型使用`字节切片`作为其内容容器，并且`会用一个字段实时地记录已读字节的计数`。

虽然我们无法直接计算出这个已读计数，但是由于它在`Buffer`值中起到的作用非常关键，所以我们很有必要去理解它。

无论是读取、写入、截断、导出还是重置，已读计数都是功能实现中的重要一环。

与`strings.Builder`类型的值一样，`Buffer`值既可以被手动扩容，也可以进行自动的扩容。除非我们完全确定后续内容所需的字节数，否则让`Buffer`值自动去扩容就好了。

`Buffer`值的扩容方法并不一定会为了获得更大的容量，替换掉现有的内容容器，而是先会**本着尽量减少内存分配和内容拷贝的原则**，对当前的内容容器进行重用。并且，只有在容量实在无法满足要求的时候，它才会去创建新的内容容器。

此外，你可能并没有想到，`Buffer`值的某些方法可能会造成内容的泄露。这主要是由于这些方法返回的结果值，在一段时期内会与其所属值的内容容器共用同一个底层数组。

如果我们有意或无意地把这些结果值传到了外界，那么外界就有可能通过它们操纵相关联`Buffer`值的内容。

这属于很严重的数据安全问题。我们一定要避免这种情况的发生。**最彻底的做法是，在传出切片这类值之前要做好隔离。比如，先对它们进行深度拷贝，然后再把副本传出去。**

## 思考题
今天的思考题是：对比strings.Builder和bytes.Buffer的String方法，并判断哪一个更高效？原因是什么？

strings.Builder的String方法更高效。因为该方法只对其所属值的内容容器（那个字节切片）做了简单的类型转换，并且直接使用了底层的值（或者说内存空间）。它的源码如下：

数组值和字符串值在底层的存储方式其实是一样的。所以从切片值到字符串值的指针值的转换可以是直截了当的。又由于字符串值是不可变的，所以这样做也是安全的。

不过，由于一些历史、结构和功能方面的原因，bytes.Buffer的String方法却不能这样做。

In [ ]:

// String returns the accumulated string.
func (b *Builder) String() string {
		return *(*string)(unsafe.Pointer(&b.buf))
}